In [1]:
import os
import random
import json
import requests
from operator import itemgetter
from typing import Union, List
from langchain_openai import AzureChatOpenAI
from langchain.agents import AgentExecutor, Tool, create_openai_tools_agent
from langchain_community.chat_message_histories import ChatMessageHistory, CosmosDBChatMessageHistory
from langchain.callbacks.manager import CallbackManager
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.runnables import ConfigurableFieldSpec, ConfigurableField
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain_core.output_parsers import StrOutputParser
from langchain.output_parsers import JsonOutputToolsParser
from langchain_core.runnables import (
    Runnable,
    RunnableLambda,
    RunnableMap,
    RunnablePassthrough,
)

#custom libraries that we will use later in the app
from common.utils import (
    SQLSearchAgent
)
from common.callbacks import StdOutCallbackHandler
from common.prompts import CUSTOM_CHATBOT_PROMPT 

from dotenv import load_dotenv
load_dotenv("credentials.env")

from IPython.display import Markdown, HTML, display 

def printmd(string):
    display(Markdown(string))

In [2]:
cb_handler = StdOutCallbackHandler()
cb_manager = CallbackManager(handlers=[cb_handler])

COMPLETION_TOKENS = 2000

# We can run the everything with GPT3.5, but try also GPT4 and see the difference in the quality of responses
# You will notice that GPT3.5 is not as reliable when using multiple sources.

# llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], 
#                       temperature=0, max_tokens=COMPLETION_TOKENS)

# Uncomment below if you want to see the answers streaming
llm = AzureChatOpenAI(deployment_name=os.environ["GPT4o_DEPLOYMENT_NAME"], temperature=0, max_tokens=COMPLETION_TOKENS, streaming=True, callback_manager=cb_manager, api_version="2024-05-01-preview")
# llm = ChatOllama(model = "llama3.2:latest", max_tokens = 128000, temperature=0, callbacks=cb_manager, disable_streaming = False)

In [3]:
sql_search = SQLSearchAgent(llm=llm, k=10, callback_manager=cb_manager,
                name="sqlsearch",
                description="useful when the questions includes the term: sqlsearch",
                verbose=False)

# Multi Agent

In [4]:
tools = [sql_search]

In [5]:
agent = create_openai_tools_agent(llm, tools, CUSTOM_CHATBOT_PROMPT)

In [6]:
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=False)

In [7]:
def get_session_history(session_id: str, user_id: str) -> CosmosDBChatMessageHistory:
    cosmos = CosmosDBChatMessageHistory(
        cosmos_endpoint=os.environ['AZURE_COSMOSDB_ENDPOINT'],
        cosmos_database=os.environ['AZURE_COSMOSDB_NAME'],
        cosmos_container=os.environ['AZURE_COSMOSDB_CONTAINER_NAME'],
        connection_string=os.environ['AZURE_COMOSDB_CONNECTION_STRING'],
        session_id=session_id,
        user_id=user_id
        )

    # prepare the cosmosdb instance
    cosmos.prepare_cosmos()
    return cosmos

In [8]:
brain_agent_executor = RunnableWithMessageHistory(
    agent_executor,
    get_session_history,
    input_messages_key="question",
    history_messages_key="history",
    history_factory_config=[
        ConfigurableFieldSpec(
            id="user_id",
            annotation=str,
            name="User ID",
            description="Unique identifier for the user.",
            default="",
            is_shared=True,
        ),
        ConfigurableFieldSpec(
            id="session_id",
            annotation=str,
            name="Session ID",
            description="Unique identifier for the conversation.",
            default="",
            is_shared=True,
        ),
    ],
)

In [9]:
# This is where we configure the session id and user id
random_session_id = "session"+ str(random.randint(1, 1000))
ramdom_user_id = "user"+ str(random.randint(1, 1000))

# random_session_id = "session428" 
# ramdom_user_id = "user325"
config={"configurable": {"session_id": random_session_id, "user_id": ramdom_user_id}}
print(random_session_id, ramdom_user_id)

session357 user525


# Test

In [ ]:
from datetime import datetime, timedelta
today = datetime.now().date()
yesterday = today - timedelta(days=1)
day_before_yesterday = today - timedelta(days=2)

# Format dates as strings
target_date_8 = day_before_yesterday.strftime("%Y-%m-%d")
target_date_9 = yesterday.strftime("%Y-%m-%d")
target_date_today = today.strftime("%Y-%m-%d")
def chat_with_agent():
    while True:
        question = input("Enter your question (or type 'exit' to quit): ")
        if question.lower() == "exit":
            print("Goodbye!")
            break
        # Add the required variables to the input
        input_data = {
            "question": question,
            "target_date_today":target_date_today,
            "target_date_9": target_date_9, 
            "target_date_8": target_date_8, 
            "history": [],  # Assuming you have a way to manage conversation history
            "agent_scratchpad": []  # Assuming you have a way to manage agent scratchpad
        }
        response = brain_agent_executor.invoke(input_data, config=config)["output"]
        printmd(response)

# Run the chat function
chat_with_agent()

Enter your question (or type 'exit' to quit):  Hi


Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Enter your question (or type 'exit' to quit):  Yes I am ready


Great! Let's draft your standup update based on your recent GitHub activities. Here's a summary:

### Accomplishments:
- Successfully closed the pull request for "Negative first response prompt working" in the DailyPulseAgentic repository.
- Integrated the git tool as a separate file and merged the changes.

### Plans:
- Continue development on the DailyPulseAgentic project, possibly addressing any feedback or new issues that arise.

### Blockers:
- No current blockers reported.

Does this draft look good to you, or would you like to make any edits or additions?

...

Enter your question (or type 'exit' to quit):  I do have a current blocker, notifications still not being sent via slack


Thank you for mentioning that. I'll update the draft to include this blocker.

### Accomplishments:
- Successfully closed the pull request for "Negative first response prompt working" in the DailyPulseAgentic repository.
- Integrated the git tool as a separate file and merged the changes.
- Completed the database setup for the DailyPulseFastAPI project.

### Plans:
- Continue development on the DailyPulseAgentic project, with a focus on transitioning the tool to a web-based interface.

### Blockers:
- Notifications are not being sent via Slack, which needs resolution.

Please confirm if this revised draft is ready for submission, or if there's anything else you'd like to adjust.

Thank you for mentioning that. I'll update the draft to include this blocker.

### Accomplishments:
- Successfully closed the pull request for "Negative first response prompt working" in the DailyPulseAgentic repository.
- Integrated the git tool as a separate file and merged the changes.
- Completed the database setup for the DailyPulseFastAPI project.

### Plans:
- Continue development on the DailyPulseAgentic project, with a focus on transitioning the tool to a web-based interface.

### Blockers:
- Notifications are not being sent via Slack, which needs resolution.

Please confirm if this revised draft is ready for submission, or if there's anything else you'd like to adjust.

Enter your question (or type 'exit' to quit):  Yes, i am ready to submit


Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with `{}`



It appears that the table you are trying to insert data into, `standup_updates`, does not exist in the database. The available table is `dailypulse`. Let's correct the table name and proceed with the insertion.

Here's the corrected insertion query:

```sql
INSERT INTO dailypulse (username, accomplishment, todo, blocker, date)
VALUES ('ashishkj21', 'Successfully closed the pull request for "Negative first response prompt working" in the DailyPulseAgentic repository. Integrated the git tool as a separate file and merged the changes. Completed the database setup for the DailyPulseFastAPI project.', 'Continue development on the DailyPulseAgentic project, with a focus on transitioning the tool to a web-based interface.', 'Notifications are not being sent via Slack, which needs resolution.', '2025-01-11');
```

I will now execute this corrected query.Agent Action: 
Invoking: `sql_db_query_checker` with `{'query': 'IN

# Sample with Positive prompt

In [11]:
def chat_with_agent():  
    while True:  
        question = input("Enter your question (or type 'exit' to quit): ")  
        if question.lower() == "exit":  
            print("Goodbye!")  
            break  
        response = brain_agent_executor.invoke({"question": question}, config=config)["output"]  
        printmd(response)  
  
# Run the chat function  
chat_with_agent()  

Enter your question (or type 'exit' to quit):  Hi


Hello! I hope you're having a great day so far. Are you ready to dive into your daily standup update, or is there something else I can assist you with today?

Hello! I hope you're having a great day so far. Are you ready to dive into your daily standup update, or is there something else I can assist you with today?

Enter your question (or type 'exit' to quit):  Yes


Fantastic! Let's get started with your daily standup update. Based on your GitHub activity from the past 24 hours, here's a draft of your update:

### Accomplishments:
- **Direct Message Functionality**: Successfully implemented and tested the direct message functionality, which is now working as expected.
- **Memory Storage Update**: Transitioned the storage of memory from a local JSON file to a database, enhancing the project's scalability and reliability.

### Plans:
- **Database Setup**: Complete the database setup for the codebase to ensure all functionalities are fully operational and integrated.

### Blockers:
- **Database Setup**: The setup for the database is not yet complete, which might delay further integration and testing of features that rely on database operations.

Please review this draft. Would you like to make any changes or add more details?

...

Enter your question (or type 'exit' to quit):  Submit it 


Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with `{}`



It appears that the table you are trying to insert data into, `standup_updates`, does not exist in the database. The available table is `dailypulse`. Let me correct the table name and proceed with the insertion.

Here's the corrected insertion query:

```sql
INSERT INTO dailypulse (username, accomplishment, todo, blocker, date)
VALUES ('ashishkj21', 'Direct Message Functionality: Successfully implemented and tested. Memory Storage Update: Transitioned storage from local JSON to database.', 'Complete the database setup for the codebase.', 'Database setup not complete, may delay further integration and testing.', '2025-01-10');
```

I will now execute this corrected query.Agent Action: 
Invoking: `sql_db_query` with `{'query': "INSERT INTO dailypulse (username, accomplishment, todo, blocker, date) VALUES ('ashishkj21', 'Direct Message Functionality: Successfully implemented and tested. Memory Storage Update: Trans

Your standup update has been successfully submitted! If there's anything else you need help with or any other updates you'd like to make, just let me know. Have a great day ahead!

Enter your question (or type 'exit' to quit):  exit


Goodbye!


# Sample with negative prompt

In [10]:
def chat_with_agent():  
    while True:  
        question = input("Enter your question (or type 'exit' to quit): ")  
        if question.lower() == "exit":  
            print("Goodbye!")  
            break  
        response = brain_agent_executor.invoke({"question": question}, config=config)["output"]  
        printmd(response)  
  
# Run the chat function  
chat_with_agent()  

Enter your question (or type 'exit' to quit):  Hi


Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update for today?

Enter your question (or type 'exit' to quit):  No


That's perfectly fine! Whenever you're ready to provide your update, just let me know. In the meantime, feel free to ask me anything else you might need assistance with. I'm here to help!

...

Enter your question (or type 'exit' to quit):  Thats all for now, thank you!


You're welcome! If you ever need assistance again, don't hesitate to reach out. Have a wonderful day!

You're welcome! If you ever need assistance again, don't hesitate to reach out. Have a wonderful day!

Enter your question (or type 'exit' to quit):  exit


Goodbye!


# Further Testing

In [10]:
printmd(brain_agent_executor.invoke({"question": "Hi"}, 
                                    config=config)["output"])

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update now, or is there something else you'd like to discuss first?

Good morning! Ready for a productive day ahead? Let's get started with your daily standup update! Are you ready to provide your update now, or is there something else you'd like to discuss first?

In [11]:
printmd(brain_agent_executor.invoke({"question": "Give me a summary of the github issues from yesterday"}, 
                                    config=config)["output"])

From the GitHub activity on January 9th, 2025, here's a summary of the issues:

1. **Issue Opened in Repository ashishkj21/DailyPulseFastAPI**
   - **Title:** DB setup not complete
   - **Details:** The issue was opened indicating that the DB setup has not been completed for the code base.
   - **Issue URL:** [DB setup not complete](https://github.com/ashishkj21/DailyPulseFastAPI/issues/6)
   - **Date:** 2025-01-09 20:59:32

This is the primary issue activity from yesterday. Would you like to delve into more details or discuss another topic?

From the GitHub activity on January 9th, 2025, here's a summary of the issues:

1. **Issue Opened in Repository ashishkj21/DailyPulseFastAPI**
   - **Title:** DB setup not complete
   - **Details:** The issue was opened indicating that the DB setup has not been completed for the code base.
   - **Issue URL:** [DB setup not complete](https://github.com/ashishkj21/DailyPulseFastAPI/issues/6)
   - **Date:** 2025-01-09 20:59:32

This is the primary issue activity from yesterday. Would you like to delve into more details or discuss another topic?

In [12]:
printmd(brain_agent_executor.invoke({"question": "Give me a summary of the github issues from 9th January"}, 
                                    config=config)["output"])

From the GitHub activity on January 9th, 2025, here's a summary of the issues:

1. **Issue Opened in Repository ashishkj21/DailyPulseFastAPI**
   - **Title:** DB setup not complete
   - **Details:** The issue was opened indicating that the DB setup has not been completed for the code base.
   - **Issue URL:** [DB setup not complete](https://github.com/ashishkj21/DailyPulseFastAPI/issues/6)
   - **Date:** 2025-01-09 20:59:32

This is the primary issue activity from that day. Would you like to delve into more details or discuss another topic?

From the GitHub activity on January 9th, 2025, here's a summary of the issues:

1. **Issue Opened in Repository ashishkj21/DailyPulseFastAPI**
   - **Title:** DB setup not complete
   - **Details:** The issue was opened indicating that the DB setup has not been completed for the code base.
   - **Issue URL:** [DB setup not complete](https://github.com/ashishkj21/DailyPulseFastAPI/issues/6)
   - **Date:** 2025-01-09 20:59:32

This is the primary issue activity from that day. Would you like to delve into more details or discuss another topic?

In [14]:
printmd(brain_agent_executor.invoke({"question": "sqlsearch Give me all details of dailypulse"}, 
                                    config=config)["output"])

Tool: sqlsearch
Agent Action: 
Invoking: `sql_db_list_tables` with `{}`



Agent Action: 
Invoking: `sql_db_schema` with `{'table_names': 'dailypulse'}`



The `dailypulse` table in the database has the following structure:

- **id** (SERIAL NOT NULL): A unique identifier for each record.
- **username** (VARCHAR(255) NOT NULL): The username of the person providing the standup update.
- **accomplishment** (TEXT): Details of tasks completed by the user since the last standup.
- **todo** (TEXT): The user's planned tasks for the current day.
- **blocker** (TEXT): Any challenges or blockers the user is currently facing.
- **date** (DATE NOT NULL): The date for which the standup update is recorded.

Here are some sample rows from the `dailypulse` table:

| id | username  | accomplishment | todo | blocker | date       |
|----|-----------|----------------|------|---------|------------|
| 1  | ashishkj21| Closed PRs: Initial memory commit and direct message functionality. | Complete DB setup, A

Here are the details from the `dailypulse` table in the database:

| id | username   | accomplishment | todo | blocker | date       |
|----|------------|----------------|------|---------|------------|
| 1  | ashishkj21 | Closed PRs: Initial memory commit and direct message functionality. | Complete DB setup, Address lack of descriptive README. | Incomplete DB setup. | 2025-01-07 |
| 2  | ashishkj21 | Successfully merged pull requests related to adding all tools to the github_toolkit.py and implement | Continue working on the database setup for the codebase. | Incomplete database setup. | 2025-01-09 |
| 3  | ashishkj21 | Successfully merged the pull request for "Direct message functionality" and "Initial memory commit" | Continue working on the database setup for the codebase, as noted in the recently opened issue. Addr | Getting blocked due to lack of tools for date and time segregation. | 2025-01-09 |

These entries provide a snapshot of your recent standup updates, including accomplishments, plans, and blockers. If you need more specific information or another query, please let me know!